In [146]:
# Untuk pengolahan data
import pandas as pd
import numpy as np

# Untuk pemodelan
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Embedding, Flatten

In [147]:
# Menyimpan masing-masing dataset kedalam variabel
gunung = pd.read_csv('GUNUNG.csv')
gunung.loc[::20]

,Nomor,﻿Page_URL,Nama,Rating,Pengunjung,Kategori,Alamat,Tentang,Ulasan
0,1,https://www.google.com/maps/place/Gn.+Anjasmor...,Gn. Anjasmoro,"4,4",(70),Puncak Gunung,"Hutan, Kec. Gondang, Kabupaten Mojokerto, Jawa...",NaN,NaN
20,21,https://www.google.com/maps/place/Gn.+Arjuna/@...,Gn. Arjuna,"4,2",(936),Puncak Gunung,"Raung Deres, Pecalukan, Kec. Prigen, Pasuruan,...",NaN,NaN
40,41,https://www.google.com/maps/place/Gn.+Lincing/...,Gn. Lincing,"4,9",(44),Puncak Gunung,"Kreweh, Gunungrejo, Kec. Singosari, Kabupaten ...",NaN,NaN
60,61,https://www.google.com/maps/place/Gn.+Jokolang...,Legenda craterului Tengger,"4,1",(17),Gunung berapi,"Area Gn. Bromo, Podokoyo, Kec. Tosari, Pasurua...",NaN,NaN


In [148]:
# Membuang kolom yang tidak dipakai
place = gunung.drop(['Ulasan', 'Tentang'], axis=1)

# Menghapus tanda kurung pada kolom "Pengunjung"
place['Pengunjung'] = place['Pengunjung'].str.replace('(','')
place['Pengunjung'] = place['Pengunjung'].str.replace(')','')

# Mengganti nilai "Nan" menjadi "1"
place['Pengunjung'] = place['Pengunjung'].fillna('1')

# Mengubah typedata kolom
place['Pengunjung'] = place['Pengunjung'].astype(float)                         # 'Pengunjung' dari 'object' jadi 'float'
place['Rating'] = place['Rating'].replace(',', '.', regex=True).astype(float)   # 'Rating' dari 'object' jadi 'float'
# Mengubah typedata kolom
place['Pengunjung'] = place['Pengunjung'].astype(int)

place.head(2)

<ipython-input-148-1a7d16931699>:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  place['Pengunjung'] = place['Pengunjung'].str.replace('(','')
<ipython-input-148-1a7d16931699>:6: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  place['Pengunjung'] = place['Pengunjung'].str.replace(')','')


,Nomor,﻿Page_URL,Nama,Rating,Pengunjung,Kategori,Alamat
0,1,https://www.google.com/maps/place/Gn.+Anjasmor...,Gn. Anjasmoro,4.4,70,Puncak Gunung,"Hutan, Kec. Gondang, Kabupaten Mojokerto, Jawa..."
1,2,https://www.google.com/maps/place/Gn.+Welirang...,Gn. Welirang,4.5,147,Puncak Gunung,"Area Hutan, Ngadirenggo/Htn, Kec. Wlingi, Kabu..."


In [149]:
place['Akumulasi'] = place['Rating']*place['Pengunjung']
place.head(2)

,Nomor,﻿Page_URL,Nama,Rating,Pengunjung,Kategori,Alamat,Akumulasi
0,1,https://www.google.com/maps/place/Gn.+Anjasmor...,Gn. Anjasmoro,4.4,70,Puncak Gunung,"Hutan, Kec. Gondang, Kabupaten Mojokerto, Jawa...",308.0
1,2,https://www.google.com/maps/place/Gn.+Welirang...,Gn. Welirang,4.5,147,Puncak Gunung,"Area Hutan, Ngadirenggo/Htn, Kec. Wlingi, Kabu...",661.5


In [150]:
place.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80 entries, 0 to 79
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Nomor       80 non-null     int64  
 1   ﻿Page_URL   80 non-null     object 
 2   Nama        80 non-null     object 
 3   Rating      62 non-null     float64
 4   Pengunjung  80 non-null     int64  
 5   Kategori    80 non-null     object 
 6   Alamat      80 non-null     object 
 7   Akumulasi   62 non-null     float64
dtypes: float64(2), int64(2), object(4)
memory usage: 5.1+ KB


In [151]:
def dict_encoder(col, data=place):

  # Mengubah kolom suatu dataframe menjadi list tanpa nilai yang sama
  unique_val = data[col].unique().tolist()

  # Melakukan encoding value kolom suatu dataframe ke angka
  val_to_val_encoded = {x: i for i, x in enumerate(unique_val)}

  # Melakukan proses encoding angka ke value dari kolom suatu dataframe
  val_encoded_to_val = {i: x for i, x in enumerate(unique_val)}
  return val_to_val_encoded, val_encoded_to_val

In [129]:
# Encoding User_Id
user_to_user_encoded, user_encoded_to_user = dict_encoder('Nomor')

# Mapping User_Id ke dataframe
place['Nama'] = place['Nomor'].map(user_to_user_encoded)

In [130]:
# Encoding Place_Id
place_to_place_encoded, place_encoded_to_place = dict_encoder('Nomor')

# Mapping Place_Id ke dataframe place
place['Nama'] = place['Nomor'].map(place_to_place_encoded)

In [142]:
# Mendapatkan jumlah user dan place
num_users, num_place = len(user_to_user_encoded), len(place_to_place_encoded)

# Mendapatkan nilai minimum dan maksimum rating
min_rating, max_rating = min(place['Rating']), max(place['Rating'])

print(f'Number of User: {num_users}, Number of Place: {num_place}, Min Rating: {min_rating}, Max Rating: {max_rating}')

Number of User: 80, Number of Place: 80, Min Rating: 1.0, Max Rating: 5.0


In [141]:
# Membuat variabel x untuk mencocokkan data user dan place menjadi satu value
x = place[['Nama', 'Nama']].values

# Membuat variabel y untuk membuat rating dari hasil
y = place['Rating'].apply(lambda x: (x - min_rating) / (max_rating - min_rating)).values

# Membagi menjadi 80% data train dan 20% data validasi
train_indices = int(0.8 * place.shape[0])
x_train, x_val, y_train, y_val = (
    x[:train_indices],
    x[train_indices:],
    y[:train_indices],
    y[train_indices:]
)

In [133]:
class RecommenderNet(tf.keras.Model):

  # Insialisasi fungsi
  def __init__(self, num_users, num_places, embedding_size, **kwargs):
    super(RecommenderNet, self).__init__(**kwargs)
    self.num_users = num_users
    self.num_places = num_places
    self.embedding_size = embedding_size
    self.user_embedding = layers.Embedding( # layer embedding user
        num_users,
        embedding_size,
        embeddings_initializer = 'he_normal',
        embeddings_regularizer = keras.regularizers.l2(1e-6)
    )
    self.user_bias = layers.Embedding(num_users, 1) # layer embedding user bias
    self.places_embedding = layers.Embedding( # layer embeddings places
        num_places,
        embedding_size,
        embeddings_initializer = 'he_normal',
        embeddings_regularizer = keras.regularizers.l2(1e-6)
    )
    self.places_bias = layers.Embedding(num_places, 1) # layer embedding places bias

  def call(self, inputs):
    user_vector = self.user_embedding(inputs[:,0]) # memanggil layer embedding 1
    user_bias = self.user_bias(inputs[:, 0]) # memanggil layer embedding 2
    places_vector = self.places_embedding(inputs[:, 1]) # memanggil layer embedding 3
    places_bias = self.places_bias(inputs[:, 1]) # memanggil layer embedding 4

    dot_user_places = tf.tensordot(user_vector, places_vector, 2)

    x = dot_user_places + user_bias + places_bias

    return tf.nn.sigmoid(x) # activation sigmoid

In [134]:
model = RecommenderNet(num_users, num_place, 50) # inisialisasi model

# model compile
model.compile(
    loss = tf.keras.losses.BinaryCrossentropy(),
    optimizer = keras.optimizers.Adam(learning_rate=0.0004),
    metrics=[tf.keras.metrics.RootMeanSquaredError()]
)

In [135]:
 class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('val_root_mean_squared_error')<0.25):
      print('Lapor! Metriks validasi sudah sesuai harapan')
      self.model.stop_training = True

In [136]:
# Memulai training

history = model.fit(
    x = x_train,
    y = y_train,
    epochs = 100,
    validation_data = (x_val, y_val),
    callbacks = [myCallback()]
)

Epoch 1/100
2/2 [==============================] - 1s 179ms/step - loss: nan - root_mean_squared_error: nan - val_loss: nan - val_root_mean_squared_error: nan
Epoch 2/100
2/2 [==============================] - 0s 26ms/step - loss: nan - root_mean_squared_error: nan - val_loss: nan - val_root_mean_squared_error: nan
Epoch 3/100
2/2 [==============================] - 0s 26ms/step - loss: nan - root_mean_squared_error: nan - val_loss: nan - val_root_mean_squared_error: nan
Epoch 4/100
2/2 [==============================] - 0s 30ms/step - loss: nan - root_mean_squared_error: nan - val_loss: nan - val_root_mean_squared_error: nan
Epoch 5/100
2/2 [==============================] - 0s 25ms/step - loss: nan - root_mean_squared_error: nan - val_loss: nan - val_root_mean_squared_error: nan
Epoch 6/100
2/2 [==============================] - 0s 26ms/step - loss: nan - root_mean_squared_error: nan - val_loss: nan - val_root_mean_squared_error: nan
Epoch 7/100
2/2 [==============================] - 

In [13]:
# Menyiapkan dataframe
place_df = place[['Nomor','Nama','Rating','Pengunjung', 'Kategori', 'Alamat']]
place_df.columns = ['id','place_name','category','rating','price', 'add']
df = place.copy()

In [14]:
# Mengambil sample user
user_id = place.Nomor.sample(1).iloc[0]
place_visited_by_user = place[place.Nomor == user_id]

In [15]:
# Membuat data lokasi yang belum dikunjungi user
place_not_visited = place_df[~place_df['id'].isin(place_visited_by_user.Nomor.values)]['id']
place_not_visited = list(
    set(place_not_visited)
    .intersection(set(place_to_place_encoded.keys()))
)

place_not_visited = [[place_to_place_encoded.get(x)] for x in place_not_visited]
user_encoder = user_to_user_encoded.get(user_id)
user_place_array = np.hstack(
    ([[user_encoder]] * len(place_not_visited), place_not_visited)
)

In [152]:
# Membuat data lokasi yang belum dikunjungi user
place_not_visited = place_df[~place_df['id'].isin(place_visited_by_user.Nomor.values)]['id']
place_not_visited = list(
    set(place_not_visited)
    .intersection(set(place_to_place_encoded.keys()))
)

place_not_visited = [[place_to_place_encoded.get(x)] for x in place_not_visited]
user_encoder = user_to_user_encoded.get(user_id)
user_place_array = np.hstack(
    ([[user_encoder]] * len(place_not_visited), place_not_visited)
)

In [154]:
# Mengambil top 3 recommendation
ratings = model.predict(user_place_array).flatten()
top_ratings_indices = ratings.argsort()[-7:][::-1]
recommended_place_ids = [
    place_encoded_to_place.get(place_not_visited[x][0]) for x in top_ratings_indices
]

print('Daftar rekomendasi untuk: {}'.format('User ' + str(user_id)))
print('===' * 15,'\n')
print('----' * 15)
print('Tempat dengan rating wisata paling tinggi dari user')
print('----' * 15)

top_place_user = (
    place_visited_by_user.sort_values(
        by='Rating',
        ascending=False
    )
    .head(5)
    .Nama.values
)


place_df_rows = place_df[place_df['id'].isin(top_place_user)]
for row in place_df_rows.itertuples():
    print(row.place_name, ':', row.category)

print('')
print('----' * 15)
print('Top 7 place recommendation')
print('----' * 15)

recommended_place = place_df[place_df['id'].isin(recommended_place_ids)]
for row, i in zip(recommended_place.itertuples(), range(1,4)):
    print(i,'.', row.place_name, '\n    ', row.category, ',', 'Harga Tiket Masuk ', row.price, ',', 'Rating Wisata ', row.rating,'\n')

print('==='*15)

3/3 [==============================] - 0s 4ms/step
Daftar rekomendasi untuk: User 77

------------------------------------------------------------
Tempat dengan rating wisata paling tinggi dari user
------------------------------------------------------------
75 : nan

------------------------------------------------------------
Top 7 place recommendation
------------------------------------------------------------
1 . 21 
     4.5 , Harga Tiket Masuk  Puncak Gunung , Rating Wisata  (4) 

2 . 22 
     4.5 , Harga Tiket Masuk  Puncak Gunung , Rating Wisata  (4) 

3 . 23 
     4.300000190734863 , Harga Tiket Masuk  Puncak Gunung , Rating Wisata  (3) 

